In [1]:
from pathlib import Path
import json
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [2]:
df_data = pd.read_csv("./traficogt_flat.csv")

In [3]:
print("Información inicial del dataset:")
print(f"Filas: {df_data.shape[0]}, Columnas: {df_data.shape[1]}")
print("\nPrimeras filas:")
print(df_data.head())
print("\nTipos de datos:")
print(df_data.dtypes)
print("\nValores nulos por columna:")
print(df_data.isnull().sum())

Información inicial del dataset:
Filas: 5604, Columnas: 44

Primeras filas:
                    id                       date             user  \
0  1834236045598056867  2024-09-12 14:22:06+00:00        traficogt   
1  1834029142565658846  2024-09-12 00:39:56+00:00     monymmorales   
2  1834039491826180424  2024-09-12 01:21:04+00:00  animaldgalaccia   
3  1833963729136091179  2024-09-11 20:20:01+00:00   EstacionDobleA   
4  1833665391698092330  2024-09-11 00:34:31+00:00       CubReserva   

               user_id           user_displayname  \
0             93938886                  traficoGT   
1            976875408                       Mony   
2  1730828822029750272           Jairo De La Nada   
3  1802661334355456000           Estación Doble A   
4  1155617398675988481  CUB Reserva Kanajuyu Z 16   

                                 user_rawDescription  \
0                    Noticias de ciudad de Guatemala   
1  Iglesia y estado son asunto separado.\nCatólic...   
2  Sancarlista. 

In [4]:
# selleccion de data usuarios
df_users = df_data[["user", "user_id", "user_displayname", "user_rawDescription", 
                   "user_created", "user_followersCount", "user_favouritesCount", 
                   "user_listedCount", "user_mediaCount", "user_location", 
                   "user_protected", "user_verified", "user_blue", "user_blueType", 
                   "user_descriptionLinks", "user_pinnedIds"]].copy()

# limpieza na
df_users = df_users.fillna("NA")

In [5]:
df_users['user'] = df_users['user'].astype(str).str.lower()
df_users['user_displayname'] = df_users['user_displayname'].astype(str).str.lower()

In [6]:
# limpieza de la descripcion del usuario
def clean_description(text):
    if text == "NA" or pd.isna(text):
        return "NA"
    
    # Convertir a string y minúsculas
    text = str(text).lower()
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remover menciones (@usuario)
    text = re.sub(r'@\w+', '', text)
    
    # Remover hashtags (#)
    text = re.sub(r'#\w+', '', text)
    
    # Remover caracteres especiales excepto espacios y letras básicas
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remover números si se considera necesario
    text = re.sub(r'\d+', '', text)
    
    # Remover espacios extra
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text if text != "" else "NA"

df_users['user_rawDescription'] = df_users['user_rawDescription'].apply(clean_description)

In [7]:
def clean_location(location):
    if location == "NA" or pd.isna(location):
        return "NA"
    
    location = str(location).lower()
    # Remover caracteres especiales
    location = re.sub(r'[^\w\s]', '', location)
    location = re.sub(r'\s+', ' ', location).strip()
    
    return location if location != "" else "NA"

df_users['user_location'] = df_users['user_location'].apply(clean_location)

In [8]:
# limpieza de la columna pinned ids
def clean_pinned_ids(value):
    if value == "NA" or pd.isna(value) or value == '':
        return "NA"
    try:
        # Intentar convertir a entero y luego a string
        return str(int(float(value)))
    except (ValueError, TypeError):
        return "NA"

df_users['user_pinnedIds'] = df_users['user_pinnedIds'].apply(clean_pinned_ids)

In [9]:
#limpiar descripction_links
def clean_description_links(value):
    if value == "NA" or pd.isna(value) or value == '':
        return "NA"
    try:
        # Si es una lista en formato string, convertir a lista limpia
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            links = eval(value)
            if isinstance(links, list):
                return ', '.join([str(link).lower() for link in links])
        return str(value).lower()
    except:
        return "NA"

df_users['user_descriptionLinks'] = df_users['user_descriptionLinks'].apply(clean_description_links)

In [10]:
# 7. Convertir columnas booleanas
bool_columns = ['user_protected', 'user_verified', 'user_blue']
for col in bool_columns:
    df_users[col] = df_users[col].apply(lambda x: True if str(x).lower() in ['true', '1', 'yes'] else False)

# %%
# 8. Convertir user_created a datetime
df_users['user_created'] = pd.to_datetime(df_users['user_created'], errors='coerce')
# Para los valores que no se pudieron convertir, usar una fecha por defecto o "NA"
df_users['user_created'] = df_users['user_created'].fillna(pd.Timestamp('2000-01-01'))

numeric_columns = ['user_followersCount', 'user_favouritesCount', 'user_listedCount', 'user_mediaCount']
for col in numeric_columns:
    df_users[col] = pd.to_numeric(df_users[col], errors='coerce').fillna(0).astype(int)

In [11]:
# Eliminar duplicados basados en user_id
print(f"Filas antes de eliminar duplicados: {len(df_users)}")
df_users = df_users.drop_duplicates(subset=['user_id'])
print(f"Filas después de eliminar duplicados: {len(df_users)}")

Filas antes de eliminar duplicados: 5604
Filas después de eliminar duplicados: 2071


In [12]:
df_users.head(10)

,user,user_id,user_displayname,user_rawDescription,user_created,user_followersCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_verified,user_blue,user_blueType,user_descriptionLinks,user_pinnedIds
0,traficogt,93938886,traficogt,noticias de ciudad de guatemala,2009-12-01 20:42:19+00:00,314368,3471,291,1292,guatemala,False,False,False,NA,0,NA
1,monymmorales,976875408,mony,iglesia y estado son asunto separado católica ...,2012-11-28 20:16:36+00:00,5502,274770,24,16644,guatemala frijolandia,False,False,False,NA,0,1794803010935431424
2,animaldgalaccia,1730828822029750272,jairo de la nada,sancarlista estudiante de filosofía ideológica...,2023-12-02 05:58:56+00:00,571,25257,2,1013,macondo,False,False,False,NA,1,1830424083143512576
3,estaciondoblea,1802661334355456000,estación doble a,si no ayuda no estorbe y no chingue,2024-06-17 11:16:06+00:00,18,30,0,11,NA,False,False,False,NA,0,NA
4,cubreserva,1155617398675988481,cub reserva kanajuyu z 16,preocupados por el medio ambiente somos una re...,2019-07-28 23:13:52+00:00,170,326,0,75,NA,False,False,False,NA,0,NA
5,alejandro2024gt,730220333576282112,alejandro alvarez,NA,2016-05-11 02:17:49+00:00,2,4,0,101,NA,False,False,False,NA,0,NA
6,julyponce21,3012525020,negrocorazón,NA,2015-02-07 15:53:17+00:00,208,32741,0,353,NA,False,False,False,NA,0,1414027750357229568
8,alexdel92022343,1311834124450172928,lanaranjamecanica,nativo de san goloteo el chiquito experto en i...,2020-10-02 01:03:54+00:00,656,7604,1,3647,guatemala,False,False,False,NA,0,1580392961195462656
9,bullgarrafuerte,1747398811792687104,josé aldana,NA,2024-01-16 23:22:01+00:00,1,246,0,1,guatemala,False,False,False,NA,0,NA
10,dbeatrizmb,3303444667,dora beatriz molina,NA,2015-08-01 14:01:39+00:00,1320,571540,8,41,NA,False,False,False,NA,0,NA


In [13]:
print("\nValores nulos por columna:")
print(df_users.isnull().sum())


Valores nulos por columna:
user                     0
user_id                  0
user_displayname         0
user_rawDescription      0
user_created             0
user_followersCount      0
user_favouritesCount     0
user_listedCount         0
user_mediaCount          0
user_location            0
user_protected           0
user_verified            0
user_blue                0
user_blueType            0
user_descriptionLinks    0
user_pinnedIds           0
dtype: int64


In [14]:
df_users.to_csv("usuarios.csv", index= False)

In [15]:
#trayendo el csv de mensajes
messages = pd.read_csv('traficogt_messages.csv')

In [16]:
# acá se hace la parte de creación de relaciones entre los nodos/usuarios usando las interacciones entre usuarios como conexiones:
# Crear DataFrame de interacciones para el grafo
# Función mejorada para extraer interacciones con filtrado de valores inválidos
def extract_interactions(row):
    interactions = []
    user = str(row.get('user', 'NA')).lower()
    user_id = str(row.get('user_id', 'NA'))
    tweet_id = str(row.get('id', 'NA'))
    
    # Validar que el usuario fuente sea válido
    if user == 'nan' or user == 'na' or not user or user.strip() == '':
        return interactions
    
    # 1. Menciones en el contenido
    if row.get('mentionedUsers_usernames', 'NA') != 'NA':
        mentions_str = str(row['mentionedUsers_usernames'])
        if mentions_str not in ['nan', 'NA', '']:
            mentions = mentions_str.split(',')
            for mention in mentions:
                mention = mention.strip().lower()
                # Filtrar menciones inválidas
                if (mention and mention != 'nan' and mention != 'na' and 
                    mention != user and not mention.startswith('nan')):
                    interactions.append({
                        'source_user': user,
                        'source_user_id': user_id,
                        'target_user': mention,
                        'interaction_type': 'mention',
                        'tweet_id': tweet_id
                    })
    
    # 2. Respuestas
    if row.get('inReplyToUser', 'NA') != 'NA':
        reply_to = str(row['inReplyToUser']).lower()
        # Filtrar respuestas inválidas
        if (reply_to and reply_to != 'nan' and reply_to != 'na' and 
            reply_to != user and not reply_to.startswith('nan')):
            interactions.append({
                'source_user': user,
                'source_user_id': user_id,
                'target_user': reply_to,
                'interaction_type': 'reply',
                'tweet_id': tweet_id
            })
    
    # 3. Citas (quotes)
    if row.get('quotedTweet_user_username', 'NA') != 'NA':
        quoted_user = str(row['quotedTweet_user_username']).lower()
        # Filtrar citas inválidas
        if (quoted_user and quoted_user != 'nan' and quoted_user != 'na' and 
            quoted_user != user and not quoted_user.startswith('nan')):
            interactions.append({
                'source_user': user,
                'source_user_id': user_id,
                'target_user': quoted_user,
                'interaction_type': 'quote',
                'tweet_id': tweet_id
            })
    
    return interactions

# Extraer todas las interacciones
all_interactions = []
for _, row in messages.iterrows():
    interactions = extract_interactions(row)
    all_interactions.extend(interactions)

# Crear DataFrame de interacciones
df_interactions = pd.DataFrame(all_interactions)

# Filtrar filas adicionales que puedan tener valores inválidos
df_interactions = df_interactions[
    (df_interactions['source_user'] != 'nan') &
    (df_interactions['source_user'] != 'na') &
    (df_interactions['target_user'] != 'nan') &
    (df_interactions['target_user'] != 'na') &
    (~df_interactions['source_user'].isna()) &
    (~df_interactions['target_user'].isna()) &
    (df_interactions['source_user'].str.strip() != '') &
    (df_interactions['target_user'].str.strip() != '')
]

# Eliminar duplicados
df_interactions = df_interactions.drop_duplicates()

print(f"Total de interacciones válidas: {len(df_interactions)}")
print(f"Usuarios fuente únicos: {df_interactions['source_user'].nunique()}")
print(f"Usuarios destino únicos: {df_interactions['target_user'].nunique()}")

# Mostrar sample de interacciones limpias
print("\nSample de interacciones válidas:")
print(df_interactions.head(10))

# Guardar CSV de interacciones limpias
df_interactions.to_csv('traficogt_interactions_clean.csv', index=False)
print("CSV de interacciones limpias guardado: traficogt_interactions_clean.csv")

Total de interacciones válidas: 15523
Usuarios fuente únicos: 1973
Usuarios destino únicos: 1117

Sample de interacciones válidas:
       source_user       source_user_id     target_user interaction_type  \
0        traficogt             93938886    monymmorales            quote   
1     monymmorales            976875408    cc_guatemala          mention   
2  animaldgalaccia  1730828822029750272  pncdeguatemala          mention   
3  animaldgalaccia  1730828822029750272     mingobguate          mention   
4  animaldgalaccia  1730828822029750272  fjimenezmingob          mention   
5  animaldgalaccia  1730828822029750272    diegoedeleon          mention   
6  animaldgalaccia  1730828822029750272  amilcarmontejo          mention   
7  animaldgalaccia  1730828822029750272       traficogt          mention   
8  animaldgalaccia  1730828822029750272  pncdeguatemala            reply   
9   estaciondoblea  1802661334355456000  amilcarmontejo          mention   

              tweet_id  
0  1834

# Análisis exploratorio

Ahora mismo tenemos 3 archivos principales:

* traficogt_messages.csv
* trafico_interactions_clean.csv
* usuarios.csv

El primer archivo es el que contiene todos los mensajes e información esencial del usuario para poder encontrar relaciones. Datos comoo el nombre del usuario y su id, además de toda la data del mensaje obviamente. Luego tenemos el de trafico_interactions_clean, este archivo contiene todas las relaciones entre usuarios, que son representados con interacciones. Algunas de las interacciones pueden ser respuestas, retweet, mención etc. Luego está el csv de usuarios, que contiene exclusivamente la información de los usuarios con el cuidado de no tener usuarios repetidos.